In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from scipy import stats
import re
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from time import time
import warnings
from sklearn.utils import resample
import statistics
import pyodbc
warnings.filterwarnings('ignore')
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('stopwords')
%matplotlib inline 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\KyleEStreepy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KyleEStreepy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KyleEStreepy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def clean(doc):
    
    def strip_html_tags(text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text

    def strip_urls(text):
        #url regex
        url_re = re.compile(r"""(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))""")
        stripped_text = url_re.sub('',text)
        return stripped_text

    def strip_emails(text):
        #email address regex
        email_re = re.compile(r'(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)')
        stripped_text = email_re.sub('',text)
        return stripped_text

    def strip_nonsense(text):
        # leave words that are at least three characters long, do not contain a number, and are no more 
        # than 17 chars long
        no_nonsense = re.findall(r'\b[a-z][a-z][a-z]+\b',text)
        stripped_text = ' '.join(w for w in no_nonsense if w != 'nan' and len(w) <= 17)
        return stripped_text
    
    doc = str(doc).lower()
    tag_free = strip_html_tags(doc)
    url_free = strip_urls(tag_free)
    email_free = strip_emails(url_free)
    normalized_1 = strip_nonsense(email_free)
    
    stop_free = " ".join([i for i in normalized_1.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(WordNetLemmatizer().lemmatize(word) for word in punc_free.split())
    
    return normalized

In [3]:
def topic_model(x):
    
    n_samples = 2000
    n_features = 1000
    n_components = 7
    n_top_words = 10
    
    def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
            message = "Topic #%d: " % topic_idx
            message += ", ".join([feature_names[i]
                                 for i in topic.argsort()[:-n_top_words - 1:-1]])
            print(message)
        print()


    print("Loading dataset...")
    t0 = time()
    data_samples = x
    print("done in %0.3fs." % (time() - t0))

    # Use tf-idf features for NMF.
    print("Extracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english',
                                    ngram_range  = (1,2))
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data_samples)
    print("done in %0.3fs." % (time() - t0))

    # Use tf (raw term count) features for LDA.
    print("Extracting tf features for LDA...")
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english',
                                   ngram_range = (1,2))
    t0 = time()
    tf = tf_vectorizer.fit_transform(data_samples)
    print("done in %0.3fs." % (time() - t0))
    print()

    # Fit the NMF model
    print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
          "n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    t0 = time()
    nmf = NMF(n_components=n_components, random_state=1,
              alpha=.1, l1_ratio=.5).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model (Frobenius norm):")
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    print_top_words(nmf, tfidf_feature_names, n_top_words)

    # Fit the NMF model
    print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
          "tf-idf features, n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    t0 = time()
    nmf = NMF(n_components=n_components, random_state=1,
              beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
              l1_ratio=.5).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    print_top_words(nmf, tfidf_feature_names, n_top_words)

    print("Fitting LDA models with tf features, "
          "n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    t0 = time()
    lda.fit(tf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in LDA model:")
    tf_feature_names = tf_vectorizer.get_feature_names()
    print_top_words(lda, tf_feature_names, n_top_words)

In [4]:
def tokenize_text(x):
    raw_text = x.tolist()

    text_data = []
    for text in raw_text:
        tokens = clean(text)
        text_data.append(tokens)
    
    return text_data

In [5]:
def lda_to_list (x):
    n_samples = 2000
    n_features = 1000
    n_components = 6
    n_top_words = 10
    #max_df=0.95, min_df=2,
    tf_vectorizer = CountVectorizer(
                                max_features=n_features,
                                stop_words='english',
                                   ngram_range = (1,2))

    tf = tf_vectorizer.fit_transform(x)
    
    lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    tf_feature_names = tf_vectorizer.get_feature_names()
    lda.fit(tf)
    temp_list =[]
    for topic_idx, topic in enumerate(lda.components_):
        #message = "Topic #%d: " % topic_idx
        message = ''
        message += ", ".join([tf_feature_names[i]
                            for i in topic.argsort()[:-n_top_words - 1:-1]])
        temp_list.append(message)
    return temp_list

## Agency 
Topic Modeling by Agency

In [88]:
df_br = pd.read_excel('data/CSS_Burden_Reduction_2020_v1.xlsx', encoding='utf-8')

In [89]:
df_br = df_br.drop(['Select your Occupational Category', 'Where do you work?', 'Survey_ID',
       'COMPONENT', 'SUB_COMPONENT', 'GRADELEVEL', 'SUP_STATUS', 'In your typical 40 hour week approximately how many hours would you classify as "low value" work?',
       'How many years have you worked for the Federal Government?',
       'Since you have worked for the Federal Government for less than 1 year, please specify the numbers of months you have worked for the Federal Government below.',
       'How many years have you worked for your current Agency?',
       'Since you have worked for your Agency for less than 1 year, please specify the number of months you have worked for your current agency below.',
       'On average how many days per month do you telework?',
       'What is the highest degree or level of education you have completed?',
       'Please select your age.',
       'Are you considering leaving your organization within the next year?'], axis=1)

In [90]:
df_br = df_br.rename(index=str, columns={
        'Please briefly describe an example of one burdensome administrative task or process which you believe is "low value"': 'low_value_text'
        })

In [91]:
df_br['AGENCY'] = df_br['AGENCY'].str.upper()

In [92]:
sentences = df_br[df_br['low_value_text'].isnull()==False]['low_value_text']

In [93]:
sid = SentimentIntensityAnalyzer()

In [94]:
df_br['AGENCY'] = df_br['AGENCY'].astype(str)
df_br['low_value_text'] = df_br['low_value_text'].astype(str)

In [95]:
df_br =df_br.loc[df_br['low_value_text'] != 'nan']

In [96]:
df_br['COMPOUND_SENT'] = df_br['low_value_text'].apply(lambda x: sid.polarity_scores(x)['compound'] if pd.isnull(x)==False else None)

In [97]:
stop = set(stopwords.words('english'))
useless_words = ['would','could','should','le','non','federal','government','agency','way','low','value','work']
exclude = set(string.punctuation) 
for word in useless_words:
    stop.add(word)

In [98]:
unique_agency = df_br['AGENCY'].unique()

In [99]:
lda_list = []

In [100]:
for agency in unique_agency:
    temp_list = []
    df_temp = df_br[df_br['AGENCY']==agency]
    sentences_temp = df_temp[df_temp['low_value_text'].isnull()==False]['low_value_text']
    return_list = lda_to_list(tokenize_text(sentences_temp))
    return_list.append(agency)  
    return_list.append(len(df_temp))
    if return_list is None:
        print(agency)
        break
    lda_list.append(return_list)

In [101]:
rand_list = lda_to_list(tokenize_text(sentences))

In [102]:
rand_list.append('ALL GOV')
rand_list.append(0)

In [103]:
lda_list.append(rand_list)

In [104]:
cols_temp = ['topic_1','topic_2','topic_3','topic_4','topic_5','topic_6','agency','comm_len']
df_lda = pd.DataFrame(lda_list,columns =cols_temp )

In [105]:
df_lda['comm_len'] = pd.to_numeric(df_lda['comm_len'])

In [106]:
df_lda_long = df_lda[df_lda['comm_len']>25]

In [107]:
df_lda_melt = pd.melt(df_lda_long,id_vars='agency',value_vars = [x for x in list(df_lda_long.columns) if x !='agency'])

In [108]:
df_lda_long.to_excel('Agency Topics.xlsx', index=False)

In [109]:
df_br.to_excel('Agency Sentiment_v2.xlsx', index=False)

In [110]:
df_br_sent = df_br[['AGENCY','COMPOUND_SENT']].groupby('AGENCY').mean()

In [114]:
df_br_sent.to_excel('Agency Sentiment.xlsx')

## Sr. Manager 
Topic Modeling by agency filtered to Supervisors Greater than GS-13

In [115]:
df_mgr = pd.read_excel('data/CSS_Burden_Reduction_2020_v1.xlsx', encoding='utf-8')

In [116]:
df_mgr = df_mgr.loc[df_mgr['SUP_STATUS'] ==1]

In [117]:
df_mgr = df_mgr.drop(['Select your Occupational Category', 'Where do you work?', 'Survey_ID',
       'COMPONENT', 'SUB_COMPONENT', 'GRADELEVEL', 'SUP_STATUS', 'In your typical 40 hour week approximately how many hours would you classify as "low value" work?',
       'How many years have you worked for the Federal Government?',
       'Since you have worked for the Federal Government for less than 1 year, please specify the numbers of months you have worked for the Federal Government below.',
       'How many years have you worked for your current Agency?',
       'Since you have worked for your Agency for less than 1 year, please specify the number of months you have worked for your current agency below.',
       'On average how many days per month do you telework?',
       'What is the highest degree or level of education you have completed?',
       'Please select your age.',
       'Are you considering leaving your organization within the next year?'], axis=1)

In [118]:
df_mgr = df_mgr.rename(index=str, columns={
        'Please briefly describe an example of one burdensome administrative task or process which you believe is "low value"': 'low_value_text'
        })

In [119]:
df_mgr['AGENCY'] = df_mgr['AGENCY'].str.upper()

In [120]:
sentences = df_mgr[df_mgr['low_value_text'].isnull()==False]['low_value_text']

In [121]:
sid = SentimentIntensityAnalyzer()

In [122]:
df_mgr['AGENCY'] = df_mgr['AGENCY'].astype(str)
df_mgr['low_value_text'] = df_mgr['low_value_text'].astype(str)

In [123]:
df_mgr =df_mgr.loc[df_mgr['low_value_text'] != 'nan']

In [124]:
df_mgr['COMPOUND_SENT'] = df_mgr['low_value_text'].apply(lambda x: sid.polarity_scores(x)['compound'] if pd.isnull(x)==False else None)

In [125]:
stop = set(stopwords.words('english'))
useless_words = ['would','could','should','le','non','federal','government','agency','way','low','value','work']
exclude = set(string.punctuation) 
for word in useless_words:
    stop.add(word)

In [126]:
unique_agency = df_mgr['AGENCY'].unique()

In [127]:
lda_list = []

In [128]:
for agency in unique_agency:
    temp_list = []
    df_temp = df_mgr[df_mgr['AGENCY']==agency]
    sentences_temp = df_temp[df_temp['low_value_text'].isnull()==False]['low_value_text']
    return_list = lda_to_list(tokenize_text(sentences_temp))
    return_list.append(agency)  
    return_list.append(len(df_temp))
    if return_list is None:
        print(agency)
        break
    lda_list.append(return_list)

In [129]:
rand_list = lda_to_list(tokenize_text(sentences))

In [130]:
rand_list.append('ALL GOV')
rand_list.append(0)

In [131]:
lda_list.append(rand_list)

In [132]:
cols_temp = ['topic_1','topic_2','topic_3','topic_4','topic_5','topic_6','agency','comm_len']
df_lda = pd.DataFrame(lda_list,columns =cols_temp )

In [133]:
df_lda['comm_len'] = pd.to_numeric(df_lda['comm_len'])

In [134]:
df_lda_long = df_lda[df_lda['comm_len']>25]

In [135]:
df_lda_melt = pd.melt(df_lda_long,id_vars='agency',value_vars = [x for x in list(df_lda_long.columns) if x !='agency'])

In [136]:
df_lda_long.to_excel('Sr. MGR Topics.xlsx', index=False)

In [137]:
df_mgr_sent = df_mgr[['AGENCY','COMPOUND_SENT']].groupby('AGENCY').mean()

In [164]:
df_mgr_sent.to_excel('Sr. MGR Sentiment.xlsx')

## GS Groups
Topic Modeling by GS Grouping

In [140]:
df_gs = pd.read_excel('data/CSS_Burden_Reduction_2020_v1.xlsx', encoding='utf-8')

In [141]:
def gs_group(df):
    gs_dict = {'GS-1' : 'GS 1-6',
                'GS-2' : 'GS 1-6',
                'GS-3' : 'GS 1-6',
                'GS-4' : 'GS 1-6',
                'GS-5' : 'GS 1-6',
                'GS-6' : 'GS 1-6',
                'GS-7' : 'GS 7-9',
                'GS-8' : 'GS 7-9',
                'GS-9' : 'GS 7-9',
                'GS-10' : 'GS 10-12',
                'GS-11' : 'GS 10-12',
                'GS-12' : 'GS 10-12',
                'GS-13' : 'GS 13',
                'GS-14' : 'GS 14',
                'GS-15' : 'GS 15',
                'Other' : 'Other',
                'SES' : 'SES',
                'SL' : 'SL & ST',
                'ST' : 'SL & ST',
                'NULL': None}
    
    df['GS GROUP'] = df['GRADELEVEL'].map(gs_dict)
    
    return df

In [142]:
df_gs = gs_group(df_gs)

In [143]:
df_gs = df_gs.drop(['Select your Occupational Category', 'Where do you work?', 'Survey_ID',
       'COMPONENT', 'SUB_COMPONENT', 'GRADELEVEL', 'SUP_STATUS', 'In your typical 40 hour week approximately how many hours would you classify as "low value" work?',
       'How many years have you worked for the Federal Government?',
       'Since you have worked for the Federal Government for less than 1 year, please specify the numbers of months you have worked for the Federal Government below.',
       'How many years have you worked for your current Agency?',
       'Since you have worked for your Agency for less than 1 year, please specify the number of months you have worked for your current agency below.',
       'On average how many days per month do you telework?',
       'What is the highest degree or level of education you have completed?',
       'Please select your age.',
       'Are you considering leaving your organization within the next year?'], axis=1)

In [144]:
df_gs = df_gs.rename(index=str, columns={
        'Please briefly describe an example of one burdensome administrative task or process which you believe is "low value"': 'low_value_text'
        })

In [145]:
sentences = df_gs[df_gs['low_value_text'].isnull()==False]['low_value_text']

In [146]:
sid = SentimentIntensityAnalyzer()

In [147]:
df_gs['GS GROUP'] = df_gs['GS GROUP'].astype(str)
df_gs['low_value_text'] = df_gs['low_value_text'].astype(str)

In [148]:
df_gs =df_gs.loc[df_gs['low_value_text'] != 'nan']

In [149]:
df_gs['COMPOUND_SENT'] = df_gs['low_value_text'].apply(lambda x: sid.polarity_scores(x)['compound'] if pd.isnull(x)==False else None)

In [150]:
stop = set(stopwords.words('english'))
useless_words = ['would','could','should','le','non','federal','government','agency','way','low','value','work']
exclude = set(string.punctuation) 
for word in useless_words:
    stop.add(word)

In [151]:
unique_group = df_gs['GS GROUP'].unique()

In [152]:
lda_list = []

In [153]:
for group in unique_group:
    temp_list = []
    df_temp = df_gs[df_gs['GS GROUP']==group]
    sentences_temp = df_temp[df_temp['low_value_text'].isnull()==False]['low_value_text']
    return_list = lda_to_list(tokenize_text(sentences_temp))
    return_list.append(group)  
    return_list.append(len(df_temp))
    if return_list is None:
        print(group)
        break
    lda_list.append(return_list)

In [154]:
rand_list = lda_to_list(tokenize_text(sentences))

In [155]:
rand_list.append('ALL GOV')
rand_list.append(0)

In [156]:
lda_list.append(rand_list)

In [157]:
cols_temp = ['topic_1','topic_2','topic_3','topic_4','topic_5','topic_6','gs group','comm_len']
df_lda = pd.DataFrame(lda_list,columns =cols_temp )

In [158]:
df_lda['comm_len'] = pd.to_numeric(df_lda['comm_len'])

In [159]:
df_lda_long = df_lda[df_lda['comm_len']>25]

In [160]:
df_lda_melt = pd.melt(df_lda_long,id_vars='gs group',value_vars = [x for x in list(df_lda_long.columns) if x !='gs group'])

In [161]:
df_lda_long.to_excel('GS Group Topics.xlsx', index=False)

In [162]:
df_gs_sent = df_gs[['GS GROUP','COMPOUND_SENT']].groupby('GS GROUP').mean()

In [165]:
df_gs_sent.to_excel('GS Group Sentiment.xlsx')